# Simulate a transcriptome with TE transcripts using polyester

1. Generate transcriptome 
 - Spliced and unspliced transcripts from GENCODE annotation
 - L1 transcripts from full-length L1HS-L1PA6 annotations in reference genome
 OR
 - L1 consensus sequences from RepBase (ask mike for file)

2. Simulate reads with polyester (see code from `./mikes_old_notebook`)
3. Quantify reads with salmon
 - build index of transcriptome (use same transcriptome from step 1)
 - quantify reads with salmon

4. Compare with original count matrix
 - figure out how to get read counts from salmon (https://salmon.readthedocs.io/en/latest/file_formats.html)